In [1]:
from option_pricers.pricing import PriceModel

In [2]:
import torch
 
class black76_torch(PriceModel):
    def __init__(self, S_0, strike, time_to_expiry, implied_vol, dividend, riskfree_rate, option_sign):
        super().__init__(S_0, strike, time_to_expiry, implied_vol, dividend, riskfree_rate, option_sign, 'Black76')
        
        self.gradient = None
        self.npv_pytorch = None
        self.greeks = None
    
    def premium(self):
         
        Phi = torch.distributions.Normal(0,1).cdf
        
        lnFK = torch.log(self.S_0 / self.K)
        si2 = (self.sigma**2/2)*self.tte
        sqrt_tte = torch.sqrt(self.tte)
        
        d1 = (lnFK + (self.r + si2)) / (self.sigma*sqrt_tte)
        d2 = d1 - self.sigma*sqrt_tte
        
        if self.option_sign == 1.:
            self.npv_pytorch = self._ert * (self.S_0 * Phi(d1) - self.K * Phi(d2))
        else:
            self.npv_pytorch = self._ert * ( self.K * Phi(-d2) - self.S_0 * Phi(-d1) )
        return self.npv_pytorch.item()
    
 
class bsm73_torch(PriceModel):
    
    '''
    this is the black scholes merton model where dividends are taken into account
    
    '''
    def __init__(self, S_0, strike, time_to_expiry, implied_vol, dividend, riskfree_rate, option_sign):
        super().__init__(S_0, strike, time_to_expiry, implied_vol, dividend, riskfree_rate, option_sign, 'Black73')
        
        self.gradient = None
        self.npv_pytorch = None
        self.greeks = None
        
        
    
    
    
    def premium(self):
         
        Phi = torch.distributions.Normal(0,1).cdf
        
        lnFK = torch.log(self.S_0 / self.K)
        si2 = (self.sigma**2/2)*self.tte
        sqrt_tte = torch.sqrt(self.tte)
        
        d1 = (lnFK + (self.r + si2)) / (self.sigma*sqrt_tte)
        d2 = d1 - self.sigma*sqrt_tte
        
        if self.option_sign == 1.:
            self.npv_pytorch = self._ert * (self.S_0 * Phi(d1) - self.K * Phi(d2))
        else:
            self.npv_pytorch = self._ert * ( self.K * Phi(-d2) - self.S_0 * Phi(-d1) )
        return self.npv_pytorch.item()
    

In [3]:
bs._ert

NameError: name 'bs' is not defined

In [4]:
option = {
            'S_0': 1000., 'strike': 2500., 
            'time_to_expiry': .5, 
            'implied_vol': .27, 
            'dividend': 0.,
            'riskfree_rate': .00, 
            'option_sign': -1.
        }

In [5]:
bs = bsm73_torch(**option)

In [6]:
bs.ert

tensor([1.], grad_fn=<ExpBackward0>)

In [7]:
bs.premium()

1500.0

In [8]:
bs.derivative  

{'underlying': 1000.0,
 'strike': 2500.0,
 'premium': 1500.0,
 'delta': -0.9999986886978149,
 'gamma': 3.275679105740892e-08,
 'rho': 0.49999934434890747,
 'vega': 0.0001133766199927777,
 'theta': 3.0611688998760656e-05,
 'strike_greek': -1.3102714468971044e-08}

In [9]:
torch.exp((bs.q - bs.r) * bs.tte)

tensor([1.], grad_fn=<ExpBackward0>)

In [17]:
 bs.tte

tensor([0.], requires_grad=True)

In [154]:
bs.derivative

{'underlying': 1000.0,
 'strike': 2500.0,
 'premium': 1473.9786376953125,
 'delta': -0.9826518893241882,
 'gamma': 1.1741434491341352e-08,
 'rho': 0.4912600517272949,
 'vega': 4.900754356640391e-05,
 'theta': 0.03440604731440544,
 'strike_greek': -4.696572020179701e-09}

In [13]:
bs.gradient

(tensor([0.2842], grad_fn=<AddBackward0>),)

In [76]:
import numpy as np
import scipy.stats as si


def PolanitzerNormsdist(x):
 PolanitzerNormsdist = si.norm.cdf(x,0.0,1.0)
 return (PolanitzerNormsdist)


def PolanitzerBlackCall(FuturePrice, StrikePrice, Maturity, RiskFreeRate, Volatility):
    d1 = (np.log(FuturePrice/StrikePrice)+(0.5*Volatility**2)*Maturity)/(Volatility*np.sqrt(Maturity))
    d2 = (np.log(FuturePrice/StrikePrice)-(0.5*Volatility**2)*Maturity)/(Volatility*np.sqrt(Maturity))
    PolanitzerBlackCall = np.exp(-RiskFreeRate*Maturity)*(FuturePrice*PolanitzerNormsdist(d1)-StrikePrice*PolanitzerNormsdist(d2))
    return(PolanitzerBlackCall)

def PolanitzerBlackPut(FuturePrice, StrikePrice, Maturity, RiskFreeRate, Volatility):
    d1 = (np.log(FuturePrice/StrikePrice)+(0.5*Volatility**2)*Maturity)/(Volatility*np.sqrt(Maturity))
    d2 = (np.log(FuturePrice/StrikePrice)-(0.5*Volatility**2)*Maturity)/(Volatility*np.sqrt(Maturity))
    PolanitzerBlackPut = np.exp(-RiskFreeRate*Maturity)*(StrikePrice*PolanitzerNormsdist(-d2)-FuturePrice*PolanitzerNormsdist(-d1))
    return(PolanitzerBlackPut)

1326.5805523713884

In [157]:
from numba import njit, float64, int16
from numpy import array
from math import log, sqrt, exp, pi
from galileo.option_pricer.compiled.math_utils import norm_cdf


@njit(float64[:](float64, float64, float64, float64, float64), cache=True, fastmath=True)
def calc_d12(s, k, b, sigma, tte):
    sqrt_tte = sqrt(tte)
    d1 = (log(s/k) + (b + sigma * sigma * 0.5) * tte) / (sigma * sqrt_tte)
    d2 = d1 - sigma * sqrt_tte
    return array((d1, d2))


@njit(float64[:](float64, float64, float64, float64, float64, int16), cache=True, fastmath=True)
def calc_nd12(s, k, b, sigma, tte, option_sign):
    ds = calc_d12(s, k, b, sigma, tte)
    nds = array((norm_cdf(option_sign * ds[0]), norm_cdf(option_sign * ds[1])))
    return nds


@njit(float64(float64, float64, float64, float64, float64, float64, int16), cache=True, fastmath=True)
def black_price(s, k, r, b, sigma, tte, option_sign):
    ebrt = exp((b - r) * tte)
    ert = exp(-r * tte)
    nds = calc_nd12(s, k, b, sigma, tte, option_sign)
    price = option_sign * (s * ebrt * nds[0] - k * ert * nds[1])
    return price

ImportError: Numba needs NumPy 1.24 or less